# Démo RL & Alignement : LineWorld

Ce notebook illustre comment un agent **naïf** optimise sa tâche (récolter un fruit) en sacrifiant un chat, et comment on peut **ré-aligner** son comportement en apprenant un modèle de récompense sur des labels humains/LLM.

---

## 1. Installation des dépendances

```bash
!pip install gymnasium stable-baselines3 scikit-learn joblib numpy matplotlib


In [ ]:
!pip install gymnasium stable-baselines3 scikit-learn joblib numpy matplotlib


In [ ]:
# Cellule: line_world.py
import numpy as np
import gymnasium as gym
from gymnasium import spaces

class LineWorld(gym.Env):
    metadata = {'render_modes': ['human']}
    def __init__(self, size: int = 5):
        super().__init__()
        self.size = size
        self.action_space = spaces.Discrete(2)
        self.observation_space = spaces.Box(0, size-1, shape=(3,), dtype=int)
        self.reset()

    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)
        # positions fixes
        self.pos_agent = 0
        self.pos_fruit = self.size - 1
        self.pos_chat  = self.size // 2
        self.done = False
        return np.array([self.pos_agent, self.pos_fruit, self.pos_chat]), {}

    def step(self, action: int):
        if self.done:
            raise RuntimeError("Appel à step() après la fin de l'épisode")
        # déplacement
        if action == 0 and self.pos_agent > 0:
            self.pos_agent -= 1
        elif action == 1 and self.pos_agent < self.size - 1:
            self.pos_agent += 1

        reward = 0
        # fruit
        if self.pos_agent == self.pos_fruit:
            reward += 1
            self.done = True
        # chat
        cat_dead = (self.pos_agent == self.pos_chat)
        # pas de pénalité naïve pour cat_dead

        return np.array([self.pos_agent, self.pos_fruit, self.pos_chat]), reward, self.done, {"cat_dead": cat_dead}

    def render(self):
        line = ["·"] * self.size
        line[self.pos_chat]  = "😺"
        line[self.pos_fruit] = "🍎"
        line[self.pos_agent] = "🤖"
        print("".join(line))


In [ ]:
# Cellule: train_naive.ipynb
from stable_baselines3 import PPO
from line_world import LineWorld

env = LineWorld(size=7)
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10_000)
model.save("ppo_naive")


In [ ]:
# Cellule: collect_labels.ipynb
import numpy as np
from stable_baselines3 import PPO
from line_world import LineWorld

env     = LineWorld(size=7)
model   = PPO.load("ppo_naive")
episodes, labels = [], []

for ep in range(1000):
    obs, _ = env.reset()
    traj, done, cat_dead = [], False, False
    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, _, done, info = env.step(action)
        traj.append(obs.copy())
        if info["cat_dead"]:
            cat_dead = True
    episodes.append(np.stack(traj))
    labels.append(0 if cat_dead else 1)

# Sauvegarde
np.save("episodes.npy", episodes, allow_pickle=True)
np.save("labels.npy", labels)
print("Labels collected:", np.bincount(labels), "episodes")


In [ ]:
# Cellule: train_reward_model.ipynb
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import joblib

episodes = np.load("episodes.npy", allow_pickle=True)
labels   = np.load("labels.npy")

# Construction des features
X, y = [], labels
for traj in episodes:
    length = len(traj)
    hits   = int((traj[:,0] == traj[:,2]).sum())
    X.append([length, hits])
X = np.array(X)

# Split & entraînement
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = MLPClassifier(hidden_layer_sizes=(16,16), max_iter=300, random_state=42)
clf.fit(X_train, y_train)
print("Acc du reward model :", clf.score(X_test, y_test))

# Sauvegarde du modèle de récompense
joblib.dump(clf, "reward_model.pkl")


In [ ]:
# Cellule: train_aligned.ipynb
import numpy as np
from stable_baselines3 import PPO
import joblib
from line_world import LineWorld

# Chargement du reward model
rm = joblib.load("reward_model.pkl")

class AlignedEnv(LineWorld):
    def step(self, action):
        obs, base_r, done, info = super().step(action)
        # features instantanées
        length = 1
        hit_cat = int(info["cat_dead"])
        p_survive = rm.predict_proba([[length, hit_cat]])[0,1]
        # récompense alignée
        reward = base_r + 0.5 * p_survive
        return obs, reward, done, info

env   = AlignedEnv(size=7)
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10_000)
model.save("ppo_aligned")


In [ ]:
# Cellule: evaluate.ipynb
import numpy as np
from stable_baselines3 import PPO
from line_world import LineWorld

def eval_model(path, n=1000):
    model = PPO.load(path)
    env   = LineWorld(size=7)
    surv, fruit = 0, 0
    for _ in range(n):
        obs, _ = env.reset()
        done, cat_dead = False, False
        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, _, done, info = env.step(action)
            if info["cat_dead"]:
                cat_dead = True
        if not cat_dead: surv += 1
        if obs[0] == obs[1]: fruit += 1
    return surv/n, fruit/n

print("Naïf    – survie chat, succès fruit :", eval_model("ppo_naive"))
print("Aligné  – survie chat, succès fruit :", eval_model("ppo_aligned"))
